In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
from os import listdir

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import cv2

2023-12-27 10:24:25.117259: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-27 10:24:25.120052: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 10:24:25.164251: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 10:24:25.164290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 10:24:25.164312: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

### Golbal variable

In [2]:
filepath = '/home/dai/7. NLP & CV/NLP/Animals'
trainDir = '/train_set/' 
testDir = '/test_set'

In [3]:
dataTrainDir = filepath + trainDir

dataTestDir = filepath + testDir

In [4]:
os.listdir(dataTestDir)

['cat', 'dog']

### Load the data

In [5]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

In [6]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [7]:
help(train_datagen.flow_from_directory)

Help on method flow_from_directory in module keras.src.preprocessing.image:

flow_from_directory(directory, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', follow_links=False, subset=None, interpolation='nearest', keep_aspect_ratio=False) method of keras.src.preprocessing.image.ImageDataGenerator instance
    Takes the path to a directory & generates batches of augmented data.
    
    Args:
      directory: string, path to the target directory. It should contain
        one subdirectory per class. Any PNG, JPG, BMP, PPM or TIF images
        inside each of the subdirectories directory tree will be included
        in the generator. See [this script](
        https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d)
        for more details.
      target_size: Tuple of integers `(height, width)`. The dimensions to
        which all images found will be

In [8]:
train = train_datagen.flow_from_directory(dataTrainDir, 
                                         class_mode = 'binary',
                                         target_size = (64,64)
                                         )

Found 12 images belonging to 2 classes.


In [9]:
test = train_datagen.flow_from_directory(dataTestDir, 
                                         class_mode = 'binary',
                                         target_size = (64,64)
                                         )

Found 6 images belonging to 2 classes.


In [10]:
train.image_data_generator

### Build the model

In [11]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3,3),
                input_shape = (64,64,3), activation = 'relu')
         )

model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 8, kernel_size = (3,3),
                 activation = 'relu')
         )

model.add(MaxPool2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [12]:
### -------------
### Model outline
### -------------

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 8)         1160      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 8)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1568)              0         
                                                                 
 dense (Dense)               (None, 16)                2

### Compile model

In [13]:
### -----------------
### Compile the model
### -----------------

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Train the model

In [14]:
model.fit(train, epochs = 10, validation_data = test,
         batch_size = 1)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6900 - accuracy: 0.5000 - val_loss: 0.7074 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 457ms/step - loss: 0.6832 - accuracy: 0.5000 - val_loss: 0.7064 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 442ms/step - loss: 0.6891 - accuracy: 0.5000 - val_loss: 0.7016 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 459ms/step - loss: 0.6610 - accuracy: 0.5833 - val_loss: 0.7068 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 429ms/step - loss: 0.6553 - accuracy: 0.6667 - val_loss: 0.7053 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 463ms/step - loss: 0.6430 - accuracy: 0.6667 - val_loss: 0.7047 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 462ms/step - loss: 0.6360 - accuracy: 0.6667 - val_loss: 0.6807 - val_accuracy: 0.5000
Epoch 8/10
1/1 [=======

In [15]:
classes = train.class_indices

classes

{'cat': 0, 'dog': 1}

In [16]:
sample = ImageDataGenerator(rescale = 1/255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

### Another method

In [17]:
test_image = load_img('/home/dai/7. NLP & CV/NLP/Animals/dog.jpeg', 
                     target_size = (64,64))

test_image = img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0)

test_image.shape

sample_1_pred = model.predict(test_image, batch_size = 1)[0][0]

for key, value in classes.items():
    
    if value == int(sample_1_pred):
        
        print(f'image is of : {key}')

1/1 [==============================] - 0s 97ms/step
image is of : cat


In [18]:
test2_image = load_img('/home/dai/7. NLP & CV/NLP/Animals/cat.jpg', 
                     target_size = (64,64))

test2_image = img_to_array(test2_image)

test2_image = np.expand_dims(test2_image, axis = 0)

test2_image.shape

sample_2_pred = model.predict(test2_image, batch_size = 1)[0][0]

for key, value in classes.items():
    
    if value == int(sample_2_pred):
        
        print(f'image is of : {key}')

1/1 [==============================] - 0s 19ms/step
image is of : cat
